Credits to [geohot](https://github.com/geohot/ai-notebooks/blob/master/mnist_gan.ipynb) for most of this code

## Model Architecture and training

In [1]:
%pip install pytorch_lightning


   ---------------------------------------- 0.0/774.6 kB ? eta -:--:--
    --------------------------------------- 10.2/774.6 kB ? eta -:--:--
    --------------------------------------- 10.2/774.6 kB ? eta -:--:--
   --- ----------------------------------- 61.4/774.6 kB 544.7 kB/s eta 0:00:02
   ---- ---------------------------------- 92.2/774.6 kB 655.4 kB/s eta 0:00:02
   ------ ------------------------------- 122.9/774.6 kB 654.9 kB/s eta 0:00:01
   ------- ------------------------------ 143.4/774.6 kB 655.8 kB/s eta 0:00:01
   -------- ----------------------------- 174.1/774.6 kB 655.4 kB/s eta 0:00:01
   -------- ----------------------------- 174.1/774.6 kB 655.4 kB/s eta 0:00:01
   ---------- --------------------------- 204.8/774.6 kB 623.6 kB/s eta 0:00:01
   ---------- --------------------------- 204.8/774.6 kB 623.6 kB/s eta 0:00:01
   ---------- --------------------------- 204.8/774.6 kB 623.6 kB/s eta 0:00:01
   ---------- --------------------------- 204.8/774.6 kB 623.6 kB

In [2]:
import random
import math
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F

import pytorch_lightning as pl

# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# uncomment for more descriptive logging 
# import logging
# FORMAT = '%(levelname)s %(name)s %(asctime)-15s %(filename)s:%(lineno)d %(message)s'
# logging.basicConfig(format=FORMAT)
# logging.getLogger().setLevel(logging.DEBUG)

c:\Users\sabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
class BaseDataModule(pl.LightningDataModule):
  def __init__(self, batch_size=32, split=0.8, *args, **kwargs):
    super().__init__()
    self.ds_X, self.ds_Y = self.get_dataset(*args, **kwargs)
    self.split = int(self.ds_X.shape[0]*split)
    self.batch_size = batch_size

  def train_dataloader(self):
    ds_X_train, ds_Y_train = self.ds_X[0:self.split], self.ds_Y[0:self.split]
    return torch.utils.data.DataLoader(list(zip(ds_X_train, ds_Y_train)), batch_size=self.batch_size)

  def val_dataloader(self):
    ds_X_test, ds_Y_test = self.ds_X[self.split:], self.ds_Y[self.split:]
    return torch.utils.data.DataLoader(list(zip(ds_X_test, ds_Y_test)), batch_size=self.batch_size)

class ReverseDataModule(BaseDataModule):
  def get_dataset(self, cnt=10000, seq_len=6):
    ds = np.random.randint(0, 10, size=(cnt, seq_len))
    return ds, ds[:, ::-1].ravel().reshape(cnt, seq_len)
  
# dataset idea from https://github.com/karpathy/minGPT/blob/master/play_math.ipynb
class AdditionDataModule(BaseDataModule):
  def get_dataset(self):
    ret = []
    for i in range(100):
      for j in range(100):
        s = i+j
        ret.append([i//10, i%10, j//10, j%10, s//100, (s//10)%10, s%10])
    ds = np.array(ret)
    return ds[:, 0:6], np.copy(ds[:, 1:])    

# this is the hardest to learn and requires 4 layers
class ParityDataModule(BaseDataModule):
  def get_dataset(self, seq_len=10):
    ds_X, ds_Y = [], []
    for i in range(2**seq_len):
      x = [int(x) for x in list(bin(i)[2:].rjust(seq_len, '0'))]
      ds_X.append(x)
      ds_Y.append((np.cumsum(x)%2).tolist())
    return np.array(ds_X), np.array(ds_Y)
  
class WikipediaDataModule(BaseDataModule):
  def get_dataset(self, seq_len=50):
    global enwik8
    if 'enwik8' not in globals():
      import requests
      enwik8_zipped = requests.get("https://data.deepai.org/enwik8.zip").content
      from zipfile import ZipFile
      import io
      enwik8 = ZipFile(io.BytesIO(enwik8_zipped)).read('enwik8')
    en = np.frombuffer(enwik8, dtype=np.uint8).astype(np.int)
    en = en[0:-seq_len+1]
    en[en>127] = 127
    return en[0:-1].reshape(-1, seq_len), en[1:].reshape(-1, seq_len)

In [4]:
def attention(queries, keys, values):
  d = queries.shape[-1]
  scores = torch.matmul(queries, keys.transpose(-2,-1))/math.sqrt(d)
  attention_weights = F.softmax(scores, dim=-1)
  return torch.matmul(attention_weights, values)

class MultiHeadAttention(nn.Module):
  def __init__(self, embed_dim, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.embed_dim, self.num_heads = embed_dim, num_heads
    assert embed_dim % num_heads == 0
    self.projection_dim = embed_dim // num_heads
    
    self.W_q = nn.Linear(embed_dim, embed_dim)
    self.W_k = nn.Linear(embed_dim, embed_dim)
    self.W_v = nn.Linear(embed_dim, embed_dim)
    self.W_o = nn.Linear(embed_dim, embed_dim)

  def transpose(self, x):
    x = x.reshape(x.shape[0], x.shape[1], self.num_heads, self.projection_dim)
    return x.permute(0, 2, 1, 3)
  
  def transpose_output(self, x):
    x = x.permute(0, 2, 1, 3)
    return x.reshape(x.shape[0], x.shape[1], self.embed_dim)
    
  def forward(self, q, k, v):
    q = self.transpose(self.W_q(q))
    k = self.transpose(self.W_k(k))
    v = self.transpose(self.W_v(v))
    output = attention(q, k, v)
    return self.W_o(self.transpose_output(output))
  
class TransformerBlock(nn.Module):
  def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
    super(TransformerBlock, self).__init__()
    self.att = MultiHeadAttention(embed_dim, num_heads)
    self.ffn = nn.Sequential(
      nn.Linear(embed_dim, ff_dim), nn.ReLU(), nn.Linear(ff_dim, embed_dim)
    )
    self.layernorm1 = nn.LayerNorm(embed_dim)
    self.layernorm2 = nn.LayerNorm(embed_dim)
    self.dropout = nn.Dropout(rate)
    
  def forward(self, x):
    x = self.layernorm1(x + self.dropout(self.att(x, x, x)))
    x = self.layernorm2(x + self.dropout(self.ffn(x)))
    return x
  
class TokenAndPositionEmbedding(nn.Module):
  def __init__(self, maxlen, vocab_size, embed_dim):
    super(TokenAndPositionEmbedding, self).__init__()
    self.token_emb = nn.Embedding(vocab_size, embed_dim)
    self.pos_emb = nn.Embedding(maxlen, embed_dim)
  def forward(self, x):
    pos = torch.arange(0, x.size(1), dtype=torch.int32, device=x.device)
    return self.token_emb(x) + self.pos_emb(pos).view(1, x.size(1), -1)

In [5]:
class LittleTransformer(pl.LightningModule):
  def __init__(self, seq_len=6, max_value=10, layer_count=2, embed_dim=128, num_heads=4, ff_dim=32):
    super().__init__()
    self.max_value = max_value
    self.model = nn.Sequential(
      TokenAndPositionEmbedding(seq_len, max_value, embed_dim),
      *[TransformerBlock(embed_dim, num_heads, ff_dim) for x in range(layer_count)],
      nn.Linear(embed_dim, max_value),
      nn.LogSoftmax(dim=-1))
    
  def forward(self, x):
    return self.model(x)
  
  def training_step(self, batch, batch_idx):
    x, y = batch
    output = self.model(x)
    loss = F.nll_loss(output.view(-1, self.max_value), y.view(-1))
    self.log("train_loss", loss)
    return loss
  
  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    pred = self.model(x).argmax(dim=2)
    val_accuracy = (pred == y).type(torch.float).mean()
    self.log("val_accuracy", val_accuracy, prog_bar=True)
  
  def configure_optimizers(self):
    if self.device.type == 'cuda':
      import apex
      return apex.optimizers.FusedAdam(self.parameters(), lr=3e-4)
    else:
      return torch.optim.Adam(self.parameters(), lr=3e-4)

In [6]:
model = LittleTransformer(seq_len=6)
trainer = pl.Trainer(enable_progress_bar=True, max_epochs=0)
data = AdditionDataModule(batch_size=64)
#data = ReverseDataModule(cnt=1000, seq_len=20)
#data = ParityDataModule(seq_len=14)
trainer.fit(model, data)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: c:\Users\sabri\OneDrive\Documents\Projects\Proof-of-Assets-Protocol\zkml\transformer-demo\lightning_logs

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 153 K 
-------------------------------------
153 K     Trainable params
0         Non-trainable params
153 K     Total params
0.613     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\sabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\sabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
`Trainer.fit` stopped: `max_epochs=0` reached.


## EZKL 

In [7]:

import os 

model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')



In [8]:

import json 
# After training, export to onnx (network.onnx) and create a data file (input.json)
x = torch.ones([1, 6], dtype=torch.long)
x = x.reshape([1, 6])

print(x)

# Flips the neural net into inference mode
model.eval()
model.to('cpu')

    # Export the model
torch.onnx.export(model,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data_json = dict(input_data = [data_array])

print(data_json)

    # Serialize data into file:
json.dump( data_json, open(data_path, 'w' ))


tensor([[1, 1, 1, 1, 1, 1]])


C:\Users\sabri\AppData\Local\Temp\ipykernel_21852\1190420391.py:3: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  scores = torch.matmul(queries, keys.transpose(-2,-1))/math.sqrt(d)


{'input_data': [[1, 1, 1, 1, 1, 1]]}


c:\Users\sabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\onnx\utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:ConstantOfShape, node name: /model/model.0/ConstantOfShape): input typestr: T1, has unsupported type: tensor(int32) (Triggered internally at ..\torch\csrc\jit\serialization\export.cpp:1421.)
  _C._check_onnx_proto(proto)


In [9]:
import ezkl 

!RUST_LOG=trace
# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True

'RUST_LOG' is not recognized as an internal or external command,
operable program or batch file.


In [10]:


res = await ezkl.calibrate_settings(data_path, model_path, settings_path, "resources")
assert res == True


In [11]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [12]:
# srs path
res = ezkl.get_srs(srs_path, settings_path)

In [13]:
# now generate the witness file 
witness_path = "gan_witness.json"

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [14]:
res = ezkl.mock(witness_path, compiled_model_path)
assert res == True

In [15]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK

res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

In [16]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

{'instances': [[[4050710420132988378, 4592888871911207431, 9635431550596092189, 3094830811046949920], [15005293851080393895, 683888734767854546, 9039942799075711334, 2497519177688949272], [4050710420132988378, 4592888871911207431, 9635431550596092189, 3094830811046949920], [15640967717350016944, 1083587571622173881, 16796151735874178085, 223793648293068984], [15640967717350016944, 1083587571622173881, 16796151735874178085, 223793648293068984], [7607641727867815613, 16556411302731023062, 17646720475294886839, 1433442205810571550], [2027943787124229356, 16103540445844736762, 3104983751552068225, 3292726628869754199], [640851020385093383, 9567146782923033812, 4330744889306511013, 1216896917296678190], [4576080355412412247, 1512624489246571065, 15361688789837648834, 3475597155891057207], [7607641727867815613, 16556411302731023062, 17646720475294886839, 1433442205810571550], [15640967717350016944, 1083587571622173881, 16796151735874178085, 223793648293068984], [2605877110841017648, 10179533

In [17]:
# VERIFY IT
res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

verified


In [19]:
sol_code_path = os.path.join('Verifier.sol')
abi_path = os.path.join('Verifier.abi')

res = ezkl.create_evm_verifier(
        vk_path,
        srs_path,
        settings_path,
        sol_code_path,
        abi_path
    )

assert res == True
assert os.path.isfile(sol_code_path)

In [ ]:
onchain_input_array = []

# using a loop
# avoiding printing last comma
formatted_output = "["
for i, value in enumerate(res["instances"]):
    for j, field_element in enumerate(value):
        onchain_input_array.append(ezkl.vecu64_to_felt(field_element))
        formatted_output += str(onchain_input_array[-1])
        if j != len(value) - 1:
            formatted_output += ", "
    formatted_output += "]"

# This will be the values you use onchain
# copy them over to remix and see if they verify
# What happens when you change a value?
print("pubInputs: ", formatted_output)
print("proof: ", "0x" + res["proof"])